In [7]:
import numpy as np

H = np.load("array/torch/HXYZ1D_loc/Jx_0.8_Jy_0.5_Jz_-0.3_hx_0.3_hz_0_lt_1/H/0.npy")

In [6]:
H

array([[ 0.075, -0.   , -0.   , -0.075],
       [-0.   , -0.075, -0.325, -0.   ],
       [-0.   , -0.325, -0.075, -0.   ],
       [-0.075, -0.   , -0.   ,  0.075]])

In [23]:
from utils import stoquastic

In [27]:
np.linalg.eigvalsh(stoquastic(H1))

array([-0.75,  0.25,  0.25,  0.25])

In [31]:
H1

array([[ 0.25, -0.  , -0.  , -0.  ],
       [-0.  , -0.25,  0.5 , -0.  ],
       [-0.  ,  0.5 , -0.25, -0.  ],
       [-0.  , -0.  , -0.  ,  0.25]])

In [30]:
stoquastic(H1)

array([[ 0.25,  0.  ,  0.  ,  0.  ],
       [ 0.  , -0.25, -0.5 ,  0.  ],
       [ 0.  , -0.5 , -0.25,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.25]])

In [21]:
np.linalg.eigvalsh(H2)

array([-0.25, -0.25, -0.25,  0.75])

In [13]:
u

array([[-0.9629321 , -0.26974391],
       [-0.26974391,  0.9629321 ]])